# RAG with Galileo, LangChain and GPT
Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions

## Step 0: Configuring the environment

This step install the necessary libraries for connecting with Galileo and the models

In [ ]:
!pip install langchain-community
!pip install langchain
!pip install langchain_openai
!pip install promptquality #Galileo
!pip install chromadb
!pip install sentence-transformers
!pip install openai
!pip install PyPDF

## Step 1: Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data form pages on the web.

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "docs/AIStudioDoc.pdf"
)
pdf_loader = PyPDFLoader(file_path)
pdf_data = pdf_loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Step 2: Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add do our vector database.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 500
CHUNK_OVERLAP = 0
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
splits = text_splitter.split_documents(pdf_data)



## Step 3: Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding = HuggingFaceEmbeddings()

vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectordb.as_retriever()

/var/folders/4z/zcddtdt53nz010hp1w2b86w40000gn/T/ipykernel_35305/1793829886.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings()
/var/folders/4z/zcddtdt53nz010hp1w2b86w40000gn/T/ipykernel_35305/1793829886.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()
/opt/anaconda3/envs/galileo/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywid

## Step 4: Model

In this example, we will use OpenAI API to connect to GPT-3.5 model. A broader range of models could be used.

In [6]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-instruct", api_key=os.environ["OPENAI_API_KEY"])

### Code to connect to Hugging Face models
#import yaml
#with open('config.yaml') as file:
    #config = yaml.safe_load(file)
#huggingfacehub_api_token = config["hf_key"]
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
#llm = HuggingFaceEndpoint(
   #huggingfacehub_api_token=huggingfacehub_api_token,
   #repo_id=repo_id,
#)



## Step 5: Chain
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from typing import List
from langchain.schema.document import Document

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

template = """You are an virtual Assistant for a Data Science platform called AI Studio. Answer the question based on the following context:

    {context}

    Question: {query}
    """
prompt = ChatPromptTemplate.from_template(template)

chain = {"context": retriever | format_docs, "query": RunnablePassthrough()} | prompt | llm | StrOutputParser()

## Step 6: Connect to Galileo
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo console to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

In [11]:
import promptquality as pq
import os

os.environ['GALILEO_API_KEY'] = os.environ['GALILEO_API_KEY_HP']
galileo_url = "https://console.hp.galileocloud.io/"
config = pq.login(galileo_url)

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as minh@rungalileo.io.


Through callbacks, we choose the metrics we want to monitor via the Galileo console. We pass a list of queries to run our created chain and log in to Galileo.


In [15]:
# Create callback handler
prompt_handler = pq.GalileoPromptCallback(
    project_name="AIStudio_RAG_Evaluate",
    #scorers=[pq.Scorers.context_adherence_luna, pq.Scorers.correctness, pq.Scorers.toxicity, pq.Scorers.sexist, pq.Scorers.chunk_attribution_utilization_luna,
    scorers=[pq.Scorers.context_adherence_plus, pq.Scorers.correctness, pq.Scorers.chunk_attribution_utilization_luna, pq.Scorers.toxicity, pq.Scorers.sexist, pq.Scorers.pii,
 ]
)

# Run your chain experiments across multiple inputs with the galileo callback
inputs = [
    "What is AI Studio",
    "How to create projects in AI Studio?",
    "How to monitor experiments?",
    "What are the different workspaces available?",
    "What, exactly, is a workspace?",
    "How to share my experiments with my team?",
    "Can I access my Git repository?",
    "Do I have access to files on my local computer?",
    "How do I access files on the cloud?",
    "Can I invite more people to my team?",
    "How do I install dependencies in AI Studio?",
    "How to set up version control in AI Studio?",
    "What kind of data formats can be imported?",
    "Can I schedule experiments to run automatically?",
    "How do I manage different environments?",
    "What are the options for running experiments?",
    "Can I collaborate with external users on projects?",
    "How can I track changes in the project?",
    "Is there a way to automate reporting?",
    "What integrations are supported?",
    "How do I update existing projects?",
    "What is the best way to organize my projects?",
    "How do I manage roles and permissions?",
    "Can I customize my workspace layout?",
    "What security features are in place?",
    "Is there a limit to the number of experiments I can run?",
    "How do I visualize experiment results?",
    "Can I deploy models directly from AI Studio?",
    "How do I back up my data?",
    "Can I use third-party libraries?",
    "Is there a way to optimize my experiments for performance?",
    "What are the best practices for data handling in AI Studio?",
    "How do I create pipelines for workflows?",
    "Can I set up notifications for completed tasks?",
    "What are the hardware requirements for running AI Studio?",
    "How do I manage resource usage?",
    "Can I import projects from other platforms?",
    "Is there support for real-time collaboration?",
    "How do I perform hyperparameter tuning?",
    "What are the limitations of the free tier?",
    "How can I extend AI Studio with custom tools?"
]

chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# publish the results of your run
prompt_handler.finish()

Processing complete!: 100%|██████████| 5/5 [00:02<00:00,  2.28it/s]   


Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Computing 🚧
cost: Computing 🚧
toxicity: Computing 🚧
sexist: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
groundedness: Computing 🚧
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/da316d6a-d39d-4537-8aaa-b9ba9efd93a5/baa628f9-4543-471c-bc5f-28387784f321?taskType=12
